In [1]:
# from translate import Translator
from langdetect import detect
from dateutil import parser
import re
import psycopg2
import pandas as pd

import translators as ts
from os import environ

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_21157/4060289784.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
conn = psycopg2.connect(
host = environ['DB_HOST'],
port = environ['DB_PORT'],
user = environ['DB_USER'],
password = environ['DB_PASSWORD'],
database = 'blogs')

In [5]:
query = r"""
    SELECT id, raw_html, language, country FROM articles
    -- WHERE article_type = 'blog'
    WHERE iso3 IS NULL
    LIMIT 500
;"""
body = pd.read_sql_query(query, conn)
body

/var/folders/tg/c_k1syfs2b9gvpkyjkcmdqx00000gn/T/ipykernel_2758/870457995.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  body = pd.read_sql_query(query, conn)


,id,raw_html,language,country
0,2275,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Türkiye
1,827,"<html lang=""ka"" dir=""ltr"" class="" js""><head>\n...",ka,UNDP საქართველო
2,7230,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,اليمن
3,742,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Representation Office\nEuropean Union
4,4918,"<html lang=""en"" data-rh=""lang""><head><title>La...",fr,None
...,...,...,...,...
495,3409,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Asia and the Pacific
496,2683,"<html lang=""ar"" dir=""rtl"" class="" js""><head>\n...",ar,المملكة العربية السعودية
497,4137,"<html lang=""da"" dir=""ltr"" class="" js""><head>\n...",da,Representation Office\nDenmark
498,7353,"<html lang=""fr"" dir=""ltr"" class="" js""><head>\n...",fr,République Centrafricaine


In [6]:
reg = r"<p>(.*?)<\/p>"

body['raw'] = body['raw_html'].apply(lambda x: ' '.join(re.findall(reg, x)))

In [10]:
def translate (row):
    lang = row['language']
    # translator = Translator(from_lang = lang, to_lang = 'en')
    # return translator.translate(row['date'].strip().upper())
    if row['country'] is not None:
        if lang != 'en':
            try:
                return ts.translate_text(row['country'].strip().lower(), from_language = lang, to_language = 'en')
            except:
                return row['country'].strip().lower()
        else:
            return row['country'].strip().lower()
    else:
        return None

In [11]:
body['country_translation'] = body.apply(translate, axis = 1)
body

,id,raw_html,language,country,raw,country_translation
0,2275,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Türkiye,The coffee cup experiment of the Accelerat...,türkiye
1,827,"<html lang=""ka"" dir=""ltr"" class="" js""><head>\n...",ka,UNDP საქართველო,ფოტოს ავტორი: დავით ხიზანიშვილი/UNDP <b>გაერ...,UNDP Georgia
2,7230,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,اليمن,"Resident Representative, UNDP Yemen Over the ...",اليمن
3,742,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Representation Office\nEuropean Union,The Funding Compact will enable the UN to res...,representation office\neuropean union
4,4918,"<html lang=""en"" data-rh=""lang""><head><title>La...",fr,None,,None
...,...,...,...,...,...,...
495,3409,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,Asia and the Pacific,"Photo: Fieni Aprilia by Greget Kalla Buana, ...",asia and the pacific
496,2683,"<html lang=""ar"" dir=""rtl"" class="" js""><head>\n...",ar,المملكة العربية السعودية,The Practice Note links the implementation of ...,Saudi Arabia
497,4137,"<html lang=""da"" dir=""ltr"" class="" js""><head>\n...",da,Representation Office\nDenmark,Igennem projektet får 10 maritime virksomheder...,Representation Office\nUnited States
498,7353,"<html lang=""fr"" dir=""ltr"" class="" js""><head>\n...",fr,République Centrafricaine,Programme des Nations Unies pour le dével...,Central African Republic


In [18]:
# body[body['country_translation'].isna()]
body[(body['country_translation'] == body['country']) & ~(body['country_translation'].isna())]

,id,raw_html,language,country,raw,country_translation
2,7230,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,اليمن,"Resident Representative, UNDP Yemen Over the ...",اليمن
39,7232,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,اليمن,"Resident Representative, UNDP Yemen Over the ...",اليمن
40,7235,"<html lang=""en"" dir=""ltr"" class="" js""><head>\n...",en,اليمن,"Resident Representative, UNDP Yemen Over the ...",اليمن
50,2508,"<html lang=""en"" data-rh=""lang""><head><title>La...",pt,,,
58,2514,"<html lang=""en"" data-rh=""lang""><head><title>La...",fr,"25540 Les, Spain",,"25540 Les, Spain"
64,2642,"<html lang=""en"" data-rh=""lang""><head><title>Gr...",hr,,,
66,2510,"<html lang=""en"" data-rh=""lang""><head><title>La...",pt,,,
73,2615,"<html lang=""en"" data-rh=""lang""><head><title>Ha...",en,,,
74,2640,"<html lang=""en"" data-rh=""lang""><head><title>Gr...",hr,,,
90,2650,"<html lang=""en"" style="""" class="" js flexbox fl...",fr,,<em>The United Nations Development Programme (...,


In [70]:
# translator = Translator(from_lang = 'es', to_lang = 'en')
# translator.translate(r'21 DE FEBREERO DE 2022')
# ts.translate_text('2 DE MAYO DE 2023', from_language = 'es', to_language = 'en')

In [71]:
cur = conn.cursor()

for i, r in body[body['datetime'].notnull()].iterrows():
    sql = """
        UPDATE articles
        SET parsed_date = %s
        WHERE id = %s
    ;"""
    cur.execute(sql, (r['datetime'], r['id']))

conn.commit()
cur.close()

In [ ]:
## CLOSE THE CONNECTION TO THE DB
conn.close()